# ADG with two real variables and Covariance inequality
**author:Alessio Benavoli**
<a href="http://www.alessiobenavoli.com"> alessiobenavoli.com </a>

We will learn how to build a PyRational **ADG (Almost Desirable Gambles)** belief model on the outcome of an experiment whose space of possibility is $\mathbb{R}^2$. 

In [1]:
%load_ext autoreload
%autoreload 2
from __future__ import absolute_import 
from PyRational.models.ADG import  ADG
from sympy import symbols,  Interval, Piecewise, Eq, exp
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

*PyRational* uses *Sympy* for symbolic mathematics. We need to define in *Sympy*  a `symbol` associated to the real variable and its relative domain  (we use Sympy `Interval` for the latter).

In [2]:
x1=symbols('x1', real=True)
x2=symbols('x2', real=True)
domain_x=[Interval(-10,10),Interval(-10,10)]

In [3]:
model = ADG([x1,x2],domain_x)
model

List of Symbols
Domain
List of desirable gambles
Avoiding sure loss?


We assume that our agent, Alice, finds the following gambles desirable.

In [4]:
G=[]
G.append( x1) 
G.append(-x1) 
G.append(x1**2-1)
G.append(1-x1**2)    
G.append( x2) 
G.append(-x2) 
G.append(x2**2-1)
G.append(1-x2**2) 

We add all these gambles to `model` as follows:

In [5]:
model.add_gambleList(G)
model

List of Symbols
Domain
List of desirable gambles
Avoiding sure loss?


In [6]:
model.Gambles

[x1, -x1, x1**2 - 1, -x1**2 + 1, x2, -x2, x2**2 - 1, -x2**2 + 1]

Note that $G$ is a list that includes all Alice's desirable gambles. We now `build` the belief model
and check if it avods sure loss:

In [7]:
optimoptions={'method_LISP': 'Cutting_plane', #'Cutting_plane', 'discretise'
                       'SolverLP':'linprog', #'linprog', 'cplex'
                       'LP_acc_constraints':1e-8,
                       'SolverNLP':'differential_evolution',
                       'NLP_alpha_cut':-0.00001,
                       'num_support_points': 150,
                       'verbose':False}
model.buildModel(options=optimoptions)
model.check_avs(options=optimoptions)
model

/home/benavoli/anaconda3/lib/python3.6/site-packages/scipy/optimize/_linprog_ip.py:1262: OptimizeWarning: Solving system with option 'sym_pos':True failed. It is normal for this to happen occasionally, especially as the solution is approached. However, if you see this frequently, consider setting option 'sym_pos' to False.
  OptimizeWarning)
/home/benavoli/anaconda3/lib/python3.6/site-packages/scipy/optimize/_linprog_ip.py:1274: OptimizeWarning: Solving system with option 'sym_pos':False failed. This may happen occasionally, especially as the solution is approached. However, if you see this frequently, your problem may be numerically challenging. If you cannot improve the formulation, consider setting 'lstsq' to True. Consider also setting `presolve` to True, if it is not already.
  OptimizeWarning)


Belief Model avoids sure  loss


List of Symbols
Domain
List of desirable gambles
Avoiding sure loss?


So Alice is **rational** or, equivalently, her set of desirable gambles is coherent.

## Inference

Assume Alice is interested in computing her maximum buying/minimum selling price for the gamble

$$
f=x_1 x_2
$$

 We can do that as follows:

In [8]:
f_range=(None,None)
f=x1*x2  
lp=model.lower_prevision(f,f_range,options=optimoptions)
up=model.upper_prevision(f,f_range,options=optimoptions)
print(lp,str("<= E[x_1 x_2] <="), up)

-0.9955920104136098 <= E[x_1 x_2] <= 0.9983673733966469


We have obtained the covariance inequality for standardised variables:

$$
|E(X_1X_2)|^2 \leq E(X_1^2) E(X_2^2).
$$

which implies

$$
-1=-\sqrt{E(X_1^2) E(X_2^2)} \leq E(X_1X_2) \leq \sqrt{E(X_1^2) E(X_2^2)}=1.
$$

## Structual judgments (independence)

Under a judgment of 'independence', if Alice finds the gambles in $G_x$ on $x_1$ desirable and the gambles
in $G_y$ on $x_2$ desirable, she should also find the gambles in $G_x \otimes G_y$ desirable

In [9]:
Gx=[]
Gx.append( Piecewise((1,True)) ) # constant 1
Gx.append( x1) 
Gx.append(-x1) 
Gx.append(x1**2-1)
Gx.append(1-x1**2)    

Gy=[]
Gy.append( Piecewise((1,True)) )
Gy.append( x2) 
Gy.append(-x2) 
Gy.append(x2**2-1)
Gy.append(1-x2**2) 

Gprod=[a*b for a in Gx for b in Gy]
Gprod

[1,
 x2,
 -x2,
 x2**2 - 1,
 -x2**2 + 1,
 x1,
 x1*x2,
 -x1*x2,
 x1*(x2**2 - 1),
 x1*(-x2**2 + 1),
 -x1,
 -x1*x2,
 x1*x2,
 -x1*(x2**2 - 1),
 -x1*(-x2**2 + 1),
 x1**2 - 1,
 x2*(x1**2 - 1),
 -x2*(x1**2 - 1),
 (x1**2 - 1)*(x2**2 - 1),
 (x1**2 - 1)*(-x2**2 + 1),
 -x1**2 + 1,
 x2*(-x1**2 + 1),
 -x2*(-x1**2 + 1),
 (-x1**2 + 1)*(x2**2 - 1),
 (-x1**2 + 1)*(-x2**2 + 1)]

In [10]:
model1 = ADG([x1,x2],domain_x)
model1.add_gambleList(Gprod)
model1.buildModel(options=optimoptions)
model1.check_avs(options=optimoptions)
model1

/home/benavoli/anaconda3/lib/python3.6/site-packages/scipy/optimize/_linprog_ip.py:1262: OptimizeWarning: Solving system with option 'sym_pos':True failed. It is normal for this to happen occasionally, especially as the solution is approached. However, if you see this frequently, consider setting option 'sym_pos' to False.
  OptimizeWarning)
/home/benavoli/anaconda3/lib/python3.6/site-packages/scipy/optimize/_linprog_ip.py:1274: OptimizeWarning: Solving system with option 'sym_pos':False failed. This may happen occasionally, especially as the solution is approached. However, if you see this frequently, your problem may be numerically challenging. If you cannot improve the formulation, consider setting 'lstsq' to True. Consider also setting `presolve` to True, if it is not already.
  OptimizeWarning)


Belief Model avoids sure  loss


List of Symbols
Domain
List of desirable gambles
Avoiding sure loss?


In [11]:
f_range=(None,None)
f=x1*x2  
lp=model1.lower_prevision(f,f_range,options=optimoptions)
up=model1.upper_prevision(f,f_range,options=optimoptions)
print(lp,str("<= E[x_1 x_2] <="), up)

/home/benavoli/anaconda3/lib/python3.6/site-packages/scipy/optimize/_linprog_ip.py:1262: OptimizeWarning: Solving system with option 'sym_pos':True failed. It is normal for this to happen occasionally, especially as the solution is approached. However, if you see this frequently, consider setting option 'sym_pos' to False.
  OptimizeWarning)
/home/benavoli/anaconda3/lib/python3.6/site-packages/scipy/optimize/_linprog_ip.py:1274: OptimizeWarning: Solving system with option 'sym_pos':False failed. This may happen occasionally, especially as the solution is approached. However, if you see this frequently, your problem may be numerically challenging. If you cannot improve the formulation, consider setting 'lstsq' to True. Consider also setting `presolve` to True, if it is not already.
  OptimizeWarning)


9.996338666551363e-09 <= E[x_1 x_2] <= -9.998929295651493e-09


This time $E[x_1 x_2]=0$ which follows from independence